# Analyze Data

## Summarize Data

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
from acquire_prepare import acquire_oil
from acquire_prepare import prep_data
import numpy as np
import pandas as pd

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import sys # used in command to make entire array print by default
import seaborn as sns
from scipy.stats import iqr

# Modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


from acquire_prepare import acquire_oil
from acquire_prepare import prep_data
from explore import xgb_rank


In [2]:
df = acquire_oil()
df = prep_data(df,'midland')

### Data Structure

In [3]:
df.shape

(6164, 61)

## Visualize Data

In [4]:
# df.head(3).T

### Attribute Histograms

In [5]:
# plot the important features #
# fig, ax = plt.subplots(figsize=(12,8))
# xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
# plt.show()

In [6]:
ranked_features, rank_df = xgb_rank(df, df.recovery)
rank_df

            feature          gain
0         direction      0.005660
1   frac_fluid_type      1.729102
2         formation      1.917078
3    frac_fluid_gpf      3.352885
4       frac_stages      3.373000
5      proppant_ppf      3.430897
6            county      3.477499
7          gor_hist      4.059082
8               tvd      4.977039
9     lateral_class      5.178928
10             oper      5.646881
11      gross_perfs      5.876164
12    mid_point_lat      5.921427
13          vintage      8.187249
14   mid_point_long     12.314726
15      oil_gravity     16.250407
16         recovery  99731.485704


,rank,feature,gain,cum_sum,cum_perc
0,1.0,recovery,99731.49,99731.49,99.91
1,2.0,oil_gravity,16.25,99747.74,99.93
2,3.0,mid_point_long,12.31,99760.05,99.94
3,4.0,vintage,8.19,99768.24,99.95
4,5.0,mid_point_lat,5.92,99774.16,99.96
5,6.0,gross_perfs,5.88,99780.04,99.96
6,7.0,oper,5.65,99785.69,99.97
7,8.0,lateral_class,5.18,99790.87,99.97
8,9.0,tvd,4.98,99795.85,99.98
9,10.0,gor_hist,4.06,99799.91,99.98


In [ ]:
ranked_features